In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
#import torch.nn.functional as F
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import namedtuple, deque
import datetime

from sklearn import preprocessing
import pandas as pd

import yfinance as yf

from torch.utils.tensorboard import SummaryWriter


In [2]:
log_dir = "stock_data_runs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(log_dir)

class DQN(nn.Module):

    def __init__(self, inputs, outputs):
        super(DQN, self).__init__()
        self.linear_relu_stack_val = nn.Sequential(
            nn.Linear(inputs, 50),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(50, 1),
        )

        self.linear_relu_stack_adv = nn.Sequential(
            nn.Linear(inputs, 50),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(50, outputs),
        )

    def forward(self, x):
        x.to(device)

        val = self.linear_relu_stack_val(x)
        adv = self.linear_relu_stack_adv(x)

        return val + adv - adv.mean()

class DQN_mid(nn.Module):

    def __init__(self, inputs, outputs):
        super(DQN, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(inputs, 2000),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(2000, 1000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(500, 200),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(200, outputs),
        )

    def forward(self, x):
        x.to(device)

        x = self.linear_relu_stack(x)
        return x
        #return self.head(x.view(x.size(0), -1))

class DQN_big(nn.Module):

    def __init__(self, inputs, outputs):
        super(DQN, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(inputs, 2000),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(2000, 2000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(2000, 2000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(2000, 2000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(2000, 2000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(2000, 2000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(2000, 2000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(2000, 2000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(2000, 2000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(2000, 1000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(500, 200),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(200, outputs),
        )

    def forward(self, x):
        x.to(device)

        x = self.linear_relu_stack(x)
        return x
        #return self.head(x.view(x.size(0), -1))

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


loss_timeline = []

In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:


time_intervall = 100

input_size = time_intervall * 3 + 1
n_actions = 3

policy_net = DQN(input_size, n_actions).to(device)
target_net = DQN(input_size, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(1000000)

steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    if steps_done % 100 == 0:
                writer.add_scalar('eps_threshold', eps_threshold, steps_done)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            #output = torch.argmax(policy_net(state))
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)


In [5]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state, dim=0)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [6]:
ORDER_COMMISSION = 1.01
random.seed(a=1337)

REWARD_SMOOTHING = 0.999
ACTION_SMOOTHING = 0.9999

avg_action_0 = 0.33
avg_action_1 = 0.33
avg_action_2 = 0.33

steps_done_amount = 100
forward_looking_timeframe = 10000

reward_array = 0

In [7]:
stock_names = ['^GDAXI','MSFT','AAPL','AMZN','DIS']
stock_datas = []
for name in stock_names:
    hist = yf.Ticker(name).history(period="max")
    stock_datas.append(hist.values)

In [19]:
BATCH_SIZE = 64
GAMMA = 0.999
EPS_START = 0.99
EPS_END = 0.001
EPS_DECAY = 10000
OPTIMIZE_MODEL = 1
TARGET_UPDATE = 100 * OPTIMIZE_MODEL

In [31]:
num_episodes = 1
num_test = 1
time_step_size = 1

for i_episode in range(num_episodes):

    print(i_episode + 1)

    for stock_data in stock_datas:

        print('new stock')

        MONEY = 1.0
        AKTIEN = 0

        time_step = time_intervall

        done = False

        buying_price = 0.0

        next_data = [buying_price]

        for i in range(time_intervall):
                #next_data.append(stock_data[time_step - time_intervall + i][0])
                next_data.append(stock_data[time_step - time_intervall + i][1])
                next_data.append(stock_data[time_step - time_intervall + i][2])
                next_data.append(stock_data[time_step - time_intervall + i][3])

        while time_step < len(stock_data):

            PREIS = stock_data[time_step][3]
            PREIS_ALT = stock_data[time_step - 1][3]
            MONEY_ALT = MONEY
            AKTIEN_ALT = AKTIEN * PREIS_ALT

            data = next_data

            state = torch.tensor([data], device=device)
            # Select and perform an action
            action = select_action(state)

            if action.item() == 0:
                #kaufen
                avg_action_0 = avg_action_0 * ACTION_SMOOTHING + (1 - ACTION_SMOOTHING) * 1
                avg_action_1 = avg_action_1 * ACTION_SMOOTHING + (1 - ACTION_SMOOTHING) * 0
                avg_action_2 = avg_action_2 * ACTION_SMOOTHING + (1 - ACTION_SMOOTHING) * 0
                if buying_price == 0:
                    AKTIEN += (MONEY * 1) / (PREIS * ORDER_COMMISSION)
                    MONEY -= MONEY * 1
                    buying_price = PREIS * ORDER_COMMISSION
                    reward = 1 - ORDER_COMMISSION
                else:
                    reward = -1.0

            elif action.item() == 1:
                #verkaufen
                avg_action_0 = avg_action_0 * ACTION_SMOOTHING + (1 - ACTION_SMOOTHING) * 0
                avg_action_1 = avg_action_1 * ACTION_SMOOTHING + (1 - ACTION_SMOOTHING) * 1
                avg_action_2 = avg_action_2 * ACTION_SMOOTHING + (1 - ACTION_SMOOTHING) * 0
                if buying_price != 0:
                    MONEY += AKTIEN * (PREIS / ORDER_COMMISSION)
                    AKTIEN = 0
                    reward = 100 * ((PREIS / ORDER_COMMISSION) - buying_price) / buying_price
                    buying_price = 0.0
                else:
                    reward = -1.0

            elif action.item() == 2:
                avg_action_0 = avg_action_0 * ACTION_SMOOTHING + (1 - ACTION_SMOOTHING) * 0
                avg_action_1 = avg_action_1 * ACTION_SMOOTHING + (1 - ACTION_SMOOTHING) * 0
                avg_action_2 = avg_action_2 * ACTION_SMOOTHING + (1 - ACTION_SMOOTHING) * 1
                reward = 0.0

            time_step += time_step_size

            if time_step >= len(stock_data):
                MONEY += AKTIEN * PREIS
                AKTIEN = 0
                break

            #reward = (MONEY + AKTIEN * PREIS) - (MONEY_ALT + AKTIEN_ALT)
            
            reward_array = reward_array * REWARD_SMOOTHING + (1 - REWARD_SMOOTHING) * reward

            if steps_done % steps_done_amount == 0:
                writer.add_scalar('avg reward', reward_array, steps_done)
                writer.add_scalar('MONEY', MONEY + AKTIEN * PREIS, steps_done)
                writer.add_scalar('avg action 0', avg_action_0, steps_done)
                writer.add_scalar('avg action 1', avg_action_1, steps_done)
                writer.add_scalar('avg action 2', avg_action_2, steps_done)

            reward = torch.tensor([reward], device=device)

            # Observe new state
            if not done:
                next_data = [buying_price]

                for i in range(time_intervall):
                        #next_data.append(stock_data[time_step - time_intervall + i][0])
                        next_data.append(stock_data[time_step - time_intervall + i][1])
                        next_data.append(stock_data[time_step - time_intervall + i][2])
                        next_data.append(stock_data[time_step - time_intervall + i][3])

                next_state = torch.tensor([next_data], device=device)
            else:
                next_state = None

            # Store the transition in memory
            memory.push(state, action, next_state, reward)

            # Perform one step of the optimization (on the policy network)
            if (steps_done) % OPTIMIZE_MODEL == 0:
                optimize_model()

            # Update the target network, copying all weights and biases in DQN
            if (steps_done) % TARGET_UPDATE == 0:
                target_net.load_state_dict(policy_net.state_dict())

            if done:
                break

print('Complete')

1
new stock
new stock
new stock
new stock
new stock
Complete


something